In [4]:
using NearestNeighbors, Dates, Statistics
using JuMP
using Gurobi
using CSV
using DataFrames;

In [11]:
function findKNN(cass_data_in, test_point_in, k)
   
    cass_data = deepcopy(cass_data_in)
    test_point = deepcopy(test_point_in)
    
    #select!(cass_data, Not(:Column1))
    cass_data = append!(cass_data, test_point)

    # normalize columns (except for year and quarter)
    
    cass_mean = mean(cass_data[!, :avgCASS]) 
    cass_std = std(cass_data[!, :avgCASS])
    
    for col in names(cass_data)
        if col == "Year" || col == "Quarter"
            continue
        end
        cass_data[!, col] = (cass_data[!, col] .- mean(cass_data[!, col])) ./ std(cass_data[!, col])
        #test_point[!, col] = (test_point[!, col] .- mean(test_point[!, col])) ./ std(test_point[!, col])
    end
    
    test_point = DataFrame(cass_data[size(cass_data)[1], :])
    X = Array(select(cass_data[1:(size(cass_data)[1]-1), :], Not(:avgCASS)))'
    kd_tree = KDTree(X)
    test_point = Array(select(test_point, Not(:avgCASS)))'
    
    id, dist = knn(kd_tree, test_point, k)
    knn_predictions = cass_data[id[1],:].avgCASS
    knn_predictions = knn_predictions .* cass_std .+ cass_mean
    return knn_predictions
end

findKNN (generic function with 1 method)

In [5]:
cass_data = CSV.read("data/data_final_train_Q1.csv", DataFrame)
test_point = CSV.read("data/data_final_test_Q1.csv", DataFrame)
select!(cass_data, Not(:Column1))
;

In [8]:
test_point

1×8 DataFrame
 Row │ Year   Quarter  OilPrice  ExChUs   USCPI    USPMI    avgCASS  PrevQuart ⋯
     │ Int64  Int64    Float64   Float64  Float64  Float64  Float64  Float64   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2020        1     61.63   6.9184  258.687     50.9  1.06467          1 ⋯
                                                                1 column omitted

In [12]:
findKNN(cass_data, test_point, 5)

5-element Vector{Float64}:
 1.262666667
 1.179333333
 1.186666667
 1.201
 1.166

In [164]:
# append both for future normalization
append!(cass_data, test_point)
;

In [165]:
test_point

1×8 DataFrame
 Row │ Year   Quarter  OilPrice  ExChUs   USCPI    USPMI    avgCASS  PrevQuart ⋯
     │ Int64  Int64    Float64   Float64  Float64  Float64  Float64  Float64   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2020        1     61.63   6.9184  258.687     50.9  1.06467          1 ⋯
                                                                1 column omitted

In [166]:
cass_data[110:114, :]

5×8 DataFrame
 Row │ Year   Quarter  OilPrice  ExChUs   USCPI    USPMI    avgCASS  PrevQuart ⋯
     │ Int64  Int64    Float64   Float64  Float64  Float64  Float64  Float64   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2019        1     56.58   6.7863  252.441     56.6  1.166            1 ⋯
   2 │  2019        2     68.58   6.7161  255.326     53.4  1.201            1
   3 │  2019        3     61.48   6.8775  255.925     51.3  1.18667          1
   4 │  2019        4     57.27   7.0961  257.387     48.5  1.10933          1
   5 │  2020        1     61.63   6.9184  258.687     50.9  1.06467          1 ⋯
                                                                1 column omitted

In [167]:
# normalize columns (except for year and quarter)
for col in names(cass_data)
    if col == "Year" || col == "Quarter"
        continue
    end
    cass_data[!, col] = (cass_data[!, col] .- mean(cass_data[!, col])) ./ std(cass_data[!, col])
    #test_point[!, col] = (test_point[!, col] .- mean(test_point[!, col])) ./ std(test_point[!, col])
end

In [168]:
cass_data[110:114, :]

5×8 DataFrame
 Row │ Year   Quarter  OilPrice  ExChUs     USCPI    USPMI      avgCASS    Pre ⋯
     │ Int64  Int64    Float64   Float64    Float64  Float64    Float64    Flo ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2019        1  0.21655   -0.543658  1.52114   0.859216   0.77388       ⋯
   2 │  2019        2  0.594278  -0.614865  1.60103   0.176187   1.11429
   3 │  2019        3  0.370789  -0.45115   1.61761  -0.272051   0.974886
   4 │  2019        4  0.23827   -0.229415  1.6581   -0.869701   0.222734
   5 │  2020        1  0.375511  -0.409664  1.69409  -0.357429  -0.211699      ⋯
                                                                1 column omitted

In [169]:
test_point = DataFrame(cass_data[size(cass_data)[1], :])

1×8 DataFrame
 Row │ Year   Quarter  OilPrice  ExChUs     USCPI    USPMI      avgCASS    Pre ⋯
     │ Int64  Int64    Float64   Float64    Float64  Float64    Float64    Flo ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2020        1  0.375511  -0.409664  1.69409  -0.357429  -0.211699      ⋯
                                                                1 column omitted

In [172]:
# get all data except the last test point we appended, and without y values
X = Array(select(cass_data[1:(size(cass_data)[1]-1), :], Not(:avgCASS)))'
#y = Array(cass_data[:, 7])
;

In [180]:
X

7×113 adjoint(::Matrix{Float64}) with eltype Float64:
 1991.0       1992.0      1992.0         …  2019.0       2019.0
    4.0          1.0         2.0               3.0          4.0
   -0.888619    -1.01736    -0.978015          0.370789     0.23827
   -1.95826     -1.88715    -1.83846          -0.45115     -0.229415
   -1.66984     -1.63938    -1.60893           1.61761      1.6581
    0.112153    -1.12584     0.00542978  …    -0.272051    -0.869701
   -1.73095     -1.60299    -1.68616           1.11298      0.975421

In [174]:
kd_tree = KDTree(X)
test_point = Array(select(test_point, Not(:avgCASS)))'


7×1 adjoint(::Matrix{Float64}) with eltype Float64:
 2020.0
    1.0
    0.3755107863964897
   -0.40966366721981
    1.694093418378432
   -0.3574294010507547
    0.23324785938813797

In [181]:
id, dist = knn(kd_tree, test_point, 5)

([[107, 106, 112, 111, 110]], [[2.8611965390782004, 2.7546977985765877, 2.4059941858155556, 1.636804566429695, 1.7212524939571345]])

In [182]:
id[1]

5-element Vector{Int64}:
 107
 106
 112
 111
 110

In [185]:
cass_data[id[1],:]

5×8 DataFrame
 Row │ Year   Quarter  OilPrice  ExChUs     USCPI    USPMI      avgCASS   Prev ⋯
     │ Int64  Int64    Float64   Float64    Float64  Float64    Float64   Floa ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2018        2  0.600888  -1.04028   1.46117   1.1367    1.71407        ⋯
   2 │  2018        1  0.520307  -0.911864  1.41814   1.39283   0.903561
   3 │  2019        3  0.370789  -0.45115   1.61761  -0.272051  0.974886
   4 │  2019        2  0.594278  -0.614865  1.60103   0.176187  1.11429
   5 │  2019        1  0.21655   -0.543658  1.52114   0.859216  0.77388        ⋯
                                                                1 column omitted